In [3]:
from iroha import Iroha, IrohaCrypto, IrohaGrpc
import ed25519

# Connection information to iroha
net = IrohaGrpc("127.0.0.1:50051")
iroha = Iroha("admin@test")

# Creating a key pair
signing_key, verifying_key = ed25519.create_keypair()

# Save the created key pair
open("iroha@test.prib","wb").write(signing_key.to_ascii(encoding="hex"))
open("iroha@test.pub","wb").write(verifying_key.to_ascii(encoding="hex"))

# Convert from binary to hexadecimal
vkey_hex = verifying_key.to_ascii(encoding="hex")

# Creating a Transaction
transfer_tx = iroha.transaction(
 [iroha.command(
    'CreateAccount',
    account_name ='iroha',
    domain_id = 'test',
    public_key = vkey_hex
 )]
)

# Transaction signature
IrohaCrypto.sign_transaction(transfer_tx,"f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70")

# Send Transaction
net.send_tx(transfer_tx)

# Check the result
for status in net.tx_status_stream(transfer_tx):
    print(status)

('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
('STATEFUL_VALIDATION_SUCCESS', 3, 0)
('COMMITTED', 5, 0)


In [7]:
get_block_query = iroha.query(
        'GetAccount',
        account_id = 'iroha@test'  
)

# Sign Query
IrohaCrypto.sign_query(get_block_query, "f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70")

# Send Query
response = net.send_query(get_block_query)

# Response output
print(response)
    

KeyboardInterrupt: 